In [3]:
from dataclasses import dataclass

In [59]:
@dataclass
class ModelConfig:
  vocab_size : int = 50257
  n_head : int = 12
  n_embed : int = 768
  n_layer : int = 12
  block_size : int = 1024
  dropout_rate : float = 0.1
  gradient_accumulation_steps : int = 16
  dtype : jnp.dtype = jnp.bfloat16


In [50]:
import jax
import jax.numpy as jnp

In [51]:
from flax import linen as nn
import jax.numpy as jnp

class CausalSelfAttention(nn.Module):
  config:ModelConfig

  @nn.compact

  def __call__(self, x, deterministic = True):
    assert len(x.shape)==3
    b, l, d = x.shape

    q = nn.Dense(self.config.n_embed)(x)
    k = nn.Dense(self.config.n_embed)(x)
    v = nn.Dense(self.config.n_embed)(x)

    q = jnp.reshape(q, (b, l, d//self.config.n_head, self.config.n_head)).astype(jnp.float32)
    k = jnp.reshape(k, (b, l, d//self.config.n_head, self.config.n_head)).astype(jnp.float32)
    v = jnp.reshape(v, (b, l, d//self.config.n_head, self.config.n_head))

    norm = jnp.sqrt(list(jnp.shape(k))[-1])

    attn = jnp.matmul(q, jnp.transpose(k, (0, 1, 3, 2))) / norm
    mask = jnp.tril(attn)

    attn = jnp.where(mask[:, :, :l, :l], attn, float("-inf")).astype(jnp.float32)
    probs = jax.nn.softmax(attn, axis=-1).astype(self.config.dtype)
    y = jnp.matmul(probs, v)
    y = jnp.reshape(y, (b, l, d))
    y = nn.Dense(self.config.n_embed)(y).astype(self.config.dtype)

    return y





In [52]:
class MLP(nn.Module):

  config : ModelConfig

  @nn.compact
  def __call__(self, x, deterministic = True):
    x=nn.Dense(self.config.n_embed*4)(x)
    x=nn.gelu(x)
    x=nn.Dropout(rate=self.config.dropout_rate)(x, deterministic=True)
    x=nn.Dense(self.config.n_embed)(x)
    x=nn.Dropout(rate=self.config.dropout_rate)(x, deterministic=True)
    return x


class Block(nn.Module):
  config : ModelConfig

  @nn.compact
  def __call__(self, x):
    x = nn.LayerNorm()(x)
    x = x + CausalSelfAttention(self.config)(x)
    x = nn.LayerNorm()(x)
    x = x + MLP(self.config)(x)
    return x

In [53]:
class GPT(nn.Module):
  config : ModelConfig

  @nn.compact

  def __call__(self, x, deterministic=True):
    B, T = x.shape
    assert T <= self.config.block_size

    pos = jnp.arange(0, T)[None]
    pos_emb = nn.Embed(self.config.block_size, self.config.n_embed)(pos)
    wte = nn.Embed(self.config.vocab_size, self.config.n_embed)
    tok_emb = wte(x)
    x = tok_emb + pos_emb

    for _ in range(self.config.n_layer):
      x = Block(self.config)(x)
    x = nn.LayerNorm()(x)
    # logits = nn.Dense(self.config.vocab_size)(x)
    logits = wte.attend(x)

    return logits


  def init(self, rng):
    tokens = jnp.zeros((1, self.config.block_size), dtype=jnp.uint16)
    params = jax.jit(super().init, static_argnums=(2,))(rng, tokens, True)
    return params



In [10]:
def count_params(params):
  p=jax.tree_util.tree_map(lambda a: a.size if isinstance(a, jnp.ndarray) else 0, params)
  return jax.tree_util.tree_reduce(lambda a, b : a+b, p)

In [60]:
config = ModelConfig()
key = jax.random.PRNGKey(0)
# model = GPT(config)
# params = model.init(key)
# count_params(params)

In [15]:
print(model)

GPT(
    # attributes
    config = ModelConfig(vocab_size=50257, n_head=12, n_embed=768, n_layer=12, block_size=1024, dropout_rate=0.1)
)


In [16]:
# from transformers import AutoModelForCausalLM

# model = AutoModelForCausalLM.from_pretrained("gpt2")

# print(model)


In [ ]:
params

{'params': {'Dense_0': {'kernel': Array([[-0.95366824,  0.43563786, -0.7954482 , -0.49190977],
          [ 0.5308177 ,  0.74109775,  0.6027838 , -0.02684463],
          [ 0.24410059,  0.44881055, -1.050442  ,  0.4932145 ]],      dtype=float32),
   'bias': Array([0., 0., 0., 0.], dtype=float32)},
  'Dense_1': {'kernel': Array([[ 0.00166371,  0.16012576],
          [ 0.09040862, -0.42028674],
          [ 0.32189375,  0.43688348],
          [-0.5580085 , -0.36031362]], dtype=float32),
   'bias': Array([0., 0.], dtype=float32)}}}

In [36]:
class DataLoader:
  def __init__(self, B, T):
    self.current_position = 0
    self.B = B
    self.T = T

    with open("/content/input.txt","r") as f:
      text = f.read()
    enc = tiktoken.get_encoding("gpt2")
    self.tokens = jnp.array(enc.encode(text))
    print(f"loaded {len(self.tokens)} tokens in the datasets" )
    print(f" 1 epoch = {len(self.tokens)//(B*T)} batches")

  def next_batch(self):
    B,T = self.B, self.T
    buf = self.tokens[self.current_position:self.current_position+B*T+1]
    x,y = jnp.reshape(buf[:-1],(B,T)), jnp.reshape(buf[1:],(B,T))
    self.current_position += B*T
    if self.current_position + B*T+1 > len(self.tokens):
      self.current_position = 0
    return x,y

In [40]:
import jax
import jax.numpy as jnp
import optax
from flax import linen as nn
from flax.training.train_state import TrainState  # <- THIS is the TrainState you need
from flax.core import FrozenDict
from typing import Tuple


def init_train_state(key, config) -> TrainState:
  model = nn.remat(
        GPT, policy=jax.checkpoint_policies.checkpoint_dots_with_no_batch_dims
        )(config)

  params = model.init(key)
  learning_rate = optax.warmup_cosine_decay_schedule(
      init_value=0.0,
      peak_value=2.5e-4,
      warmup_steps= 2000,
      decay_steps= 150000,
      end_value = 1e-5,
)
  optimizer = optax.chain(
      optax.clip_by_global_norm(1.0),
      optax.adamw(learning_rate, b1=0.9, b2=0.95,  weight_decay=1e-2)

  if config.gradient_accumulation_steps>1:
    optimizer = optax.MultiSteps(
          optimizer, every_k_schedule=config.gradient_accumulation_steps
    )
)
  train_state = TrainState.create(
        apply_fn=model.apply,
        params=params,
        tx=optimizer)
  return train_state

@jax.jit
def train_step(state: TrainState, x: jnp.ndarray, y: jnp.ndarray) -> Tuple[jnp.ndarray, TrainState]:

  def loss_fn(params: FrozenDict) -> jnp.ndarray:

      logits = state.apply_fn(params, x, False)
      loss = optax.softmax_cross_entropy_with_integer_labels(logits, y).mean()
      print(loss)
      return loss

  loss, grads = jax.value_and_grad(loss_fn, has_aux=False)(state.params)
  new_state = state.apply_gradients(grads=grads)
  return loss, new_state

In [45]:
import tiktoken
import time, math
rng = jax.random.PRNGKey(0)
train_state = init_train_state(rng, ModelConfig())
train_steps = 50
data_loader = DataLoader(B=4, T=128)
x, y = data_loader.next_batch()
for step in range(train_steps):
  t0 = time.time()
  loss, train_state = train_step(train_state, x, y)
  t1 = time.time()
  dt = t1-t0

  tokens_processed = data_loader.B * data_loader.T
  tokens_per_sec = tokens_processed/dt

  print(f"step {step}/{train_steps} | loss : {loss:.4f} | dt {dt*1000 :.2f}ms | token/sec = {tokens_per_sec:.3f}")

loaded 338025 tokens in the datasets
 1 epoch = 660 batches
Traced<ShapedArray(float32[])>with<JVPTrace> with
  primal = Traced<ShapedArray(float32[])>with<DynamicJaxprTrace>
  tangent = Traced<ShapedArray(float32[])>with<JaxprTrace> with
    pval = (ShapedArray(float32[]), None)
    recipe = JaxprEqnRecipe(eqn_id=<object object at 0x7bf494721970>, in_tracers=(Traced<ShapedArray(float32[4,128]):JaxprTrace>,), out_tracer_refs=[<weakref at 0x7bf4cd71e9d0; to 'JaxprTracer' at 0x7bf4cd71d630>], out_avals=[ShapedArray(float32[])], primitive=pjit, params={'jaxpr': { lambda ; a:f32[4,128]. let
    b:f32[] = reduce_sum[axes=(0, 1)] a
    c:f32[] = div b 512.0
  in (c,) }, 'in_shardings': (UnspecifiedValue,), 'out_shardings': (UnspecifiedValue,), 'in_layouts': (None,), 'out_layouts': (None,), 'donated_invars': (False,), 'ctx_mesh': None, 'name': '_mean', 'keep_unused': False, 'inline': True, 'compiler_options_kvs': ()}, effects=set(), source_info=<jax._src.source_info_util.SourceInfo object at 

In [38]:
data_loader = DataLoader(B=4, T=128)
x, y = data_loader.next_batch()

loaded 338025 tokens in the datasets
 1 epoch = 660 batches


In [61]:
for step in range(train_steps):
  t0 = time.time()
  for _ in range(config.gradient_accumulation_steps):
    x,y = data_loader.next_batch()
    loss, train_state = train_step(train_state, x, y)
  t1 = time.time()
  dt = t1 - t0
  tokens_processed = data_loader.B * data_loader.T * config.gradient_accumulation_steps
  tokens_per_sec = tokens_processed / dt
  print(f"step {step}/{train_steps} | loss: {loss:4f} |  dt: {dt*1000:.2f}ms | token/sec = {tokens_per_sec:.2f}")

step 0/50 | loss: 6.206775 |  dt: 454.12ms | token/sec = 18039.17
step 1/50 | loss: 6.237356 |  dt: 695.14ms | token/sec = 11784.76
step 2/50 | loss: 5.584978 |  dt: 460.25ms | token/sec = 17799.06
step 3/50 | loss: 6.498955 |  dt: 472.78ms | token/sec = 17327.16
step 4/50 | loss: 5.995944 |  dt: 472.78ms | token/sec = 17327.21
step 5/50 | loss: 6.367478 |  dt: 464.63ms | token/sec = 17631.38
step 6/50 | loss: 5.633333 |  dt: 464.19ms | token/sec = 17647.76
step 7/50 | loss: 6.112411 |  dt: 470.71ms | token/sec = 17403.58
step 8/50 | loss: 6.181900 |  dt: 470.95ms | token/sec = 17394.65
step 9/50 | loss: 6.271253 |  dt: 465.72ms | token/sec = 17590.06
step 10/50 | loss: 5.743571 |  dt: 469.21ms | token/sec = 17459.28
step 11/50 | loss: 5.862834 |  dt: 472.38ms | token/sec = 17341.98
step 12/50 | loss: 6.046236 |  dt: 470.13ms | token/sec = 17425.07
step 13/50 | loss: 6.785887 |  dt: 466.11ms | token/sec = 17575.13
step 14/50 | loss: 5.924488 |  dt: 469.74ms | token/sec = 17439.34
step 